# Γιάννης Δαλιάνης
# 1115201700027
# Homework 2
# Άσκηση 5

In [ ]:
import os
import re
from google.colab import drive
import pandas as pd

import nltk
nltk.download('stopwords')

from sklearn.preprocessing import LabelEncoder, MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, roc_curve, auc

import warnings
warnings.filterwarnings("ignore")

import random
import numpy as np

import spacy
# nlp = spacy.load('en')
nlp = spacy.load("en_core_web_sm")

from matplotlib import pyplot as plt

import torch.nn.functional as F

import time

import torch   
from torchtext import data, datasets
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

Drop uneccessary columns.

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

tweets = pd.read_csv('/content/gdrive/My Drive/giannikos/SentimentTweets.csv', index_col=0)
print(tweets.columns)
tweets = tweets.drop(['id', 'date', 'flag', 'user'], axis=1)
tweets.rename(columns={'target': 'label'}, inplace = True)

tweets

In [ ]:
# tweets = tweets.sample(n = 1000)

Clean tweet text, remove stopwords and save to new csv file.

In [ ]:
tweets = tweets.apply(lambda x: x.astype(str).str.lower())

def cleanText(text):
    text = text.str.replace(r'&amp;', '&')            # Replace '&amp;' with '&'##################
    text = text.str.replace(r'RT[\s]+', '')                                             # Removing RT
    text = text.str.replace(r'#.*?(?=\s|$)', '')                                        # remove hashtags and mentions
    text = text.str.replace(r'@.*?(?=\s|$)', '')
    text = text.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)  # remove urls
    text = text.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
    text = text.str.replace(r"\'re", " are")                                             # Change 're to 'are'
    text = text.str.replace(r"\'t", " not")                                             # Change 't to 'not'
    text = text.replace(r'\\n',' ', regex=True)                                         # remove newlines and other special characters
    text = text.replace(r'\\u',' ', regex=True)
    text = text.replace(r'\\x',' ', regex=True)
    text = text.str.replace('\d+', '')                                                  # remove all numbers
    text = text.str.replace(r'\b(\w{1,2})\b', '')                                       # remove words with 2 or 1 letter only
    text = text.str.replace('[^\w\s]','')                                               # remove punctuations
    text = text.apply(lambda x: re.sub(' +', ' ', x))                                   # replace multiple whitespaces
    return text

def remove_stops(row):
  stops = nltk.corpus.stopwords.words("english")
  meaningful_words = [w for w in row if w not in stops]
  return meaningful_words

def rejoin_words(row):
  joined_words = ( " ".join(row))
  return joined_words

tweets['text'] = cleanText(tweets['text'])

# erase empty lines
nan_value = float("NaN")
tweets['text'].replace(" ", "", inplace=True)
tweets['text'].replace("", nan_value, inplace=True)
tweets.dropna(subset=['text'], inplace=True)

# 0 will be for negative and 1 for positive
tweets['label'] = tweets['label'].astype(int)
tweets['label'].replace(4, 1, inplace=True)

# takes some time
tweets["text"] = tweets["text"].str.split()
tweets['text'] = tweets['text'].apply(lambda x: remove_stops(x))
tweets['text'] = tweets['text'].apply(lambda x: rejoin_words(x))

# erase empty lines
nan_value = float("NaN")
tweets['text'].replace(" ", "", inplace=True)
tweets['text'].replace("", nan_value, inplace=True)
tweets.dropna(subset=['text'], inplace=True)

tweets.reset_index(drop = True, inplace = True)

print(tweets.columns)

tweets["label"].replace({0: "neg", 1: "pos"}, inplace=True)

trainTWEETS, testTWEETS = train_test_split(tweets, test_size=0.2)

trainTWEETS.to_csv(r'/content/gdrive/My Drive/giannikos/trainTWEETS.csv')
testTWEETS.to_csv(r'/content/gdrive/My Drive/giannikos/testTWEETS.csv')

tweets

In [ ]:
tmpTRY = pd.read_csv('/content/gdrive/My Drive/giannikos/trainTWEETS.csv', index_col=0)
tmpTRY

In [ ]:
tmpTRY = pd.read_csv('/content/gdrive/My Drive/giannikos/testTWEETS.csv', index_col=0)

# testTRY1 = tmpTRY.copy(deep=True)
testTRY2 = tmpTRY.copy(deep=True)
testTRY3 = tmpTRY.copy(deep=True)

tmpTRY

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [ ]:
def percentage(part, whole):
  return 100 * float(part)/float(whole)

In [ ]:
if(torch.cuda.is_available()):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

In [ ]:
# start_timeTOTAL = time.time()
# TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
# LABEL = data.LabelField(dtype = torch.float, is_target=True, unk_token=None, batch_first=True)
# # (None, None) for Unamed 0
# fields = [(None, None), ('label', LABEL), ('text',TEXT)]
# training_data = data.TabularDataset(
#                           path =  '/content/gdrive/My Drive/DI/trainTWEETS.csv',
#                           format = 'csv',
#                           fields = fields,
#                           skip_header = True
#                   )
# test_data = data.TabularDataset(
#                           path =  '/content/gdrive/My Drive/DI/testTWEETS.csv',
#                           format = 'csv',
#                           fields = fields,
#                           skip_header = True
#                   )
# for i in range(5):
#   print(vars(training_data.examples[i]))
# train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))
# print(f'Number of training examples: {len(train_data)}')
# print(f'Number of validation examples: {len(valid_data)}')
# print(f'Number of test examples: {len(test_data)}')
# #initialize glove embeddings
# MAX_VOCAB_SIZE = 25_000
# TEXT.build_vocab(
#         train_data,
#         max_size = MAX_VOCAB_SIZE,
#         vectors = "glove.6B.100d",
#         unk_init = torch.Tensor.normal_,
#         # min_freq=4,
#     )
# LABEL.build_vocab(train_data)
# print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
# print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
# #Commonly used words
# print(TEXT.vocab.freqs.most_common(10))
# untill=0
# for item in TEXT.vocab.stoi.items():
#     print(item)
#     untill+=1
#     if(untill==10):
#       break
# print(TEXT.vocab.itos[:10])
# print(LABEL.vocab.stoi)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
# BATCH_SIZE = 64
# #Load an iterator
# train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
#       (train_data, valid_data, test_data), 
#       batch_size = BATCH_SIZE,
#       sort_key = lambda x: len(x.text),
#       sort_within_batch=True,
#       device = device
#     )
# class RNN(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
#                  bidirectional, dropout, pad_idx):
#         super().__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
#         self.rnn = nn.LSTM(embedding_dim, 
#                            hidden_dim, 
#                            num_layers=n_layers, 
#                            bidirectional=bidirectional, 
#                            dropout=dropout)
#         self.fc = nn.Linear(hidden_dim * 2, output_dim)
#         self.dropout = nn.Dropout(dropout)
#     def forward(self, text, text_lengths):
#         embedded = self.dropout(self.embedding(text))
#         #pack sequence
#         if(torch.cuda.is_available()):
#             packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths = text_lengths.cpu())
#         else:
#             packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
#         packed_output, (hidden, cell) = self.rnn(packed_embedded)
#         #unpack sequence
#         output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
#         hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
#         return self.fc(hidden)
# #define hyperparameters
# INPUT_DIM = len(TEXT.vocab)
# EMBEDDING_DIM = 100
# N_FILTERS = 100
# HIDDEN_DIM = 256
# OUTPUT_DIM = 1
# N_LAYERS = 2
# BIDIRECTIONAL = True
# DROPOUT = 0.5
# N_EPOCHS = 5
# PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
# model = RNN(
#     INPUT_DIM, 
#     EMBEDDING_DIM, 
#     HIDDEN_DIM, 
#     OUTPUT_DIM, 
#     N_LAYERS, 
#     BIDIRECTIONAL, 
#     DROPOUT, 
#     PAD_IDX
#   )
# print(model)
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(f'The model has {count_parameters(model):,} trainable parameters')
# pretrained_embeddings = TEXT.vocab.vectors
# print(pretrained_embeddings.shape)
# model.embedding.weight.data.copy_(pretrained_embeddings)
# UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
# model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
# model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
# print(model.embedding.weight.data)
# optimizer = optim.Adam(model.parameters())
# criterion = nn.BCEWithLogitsLoss()
# #push to cuda if available
# model = model.to(device)
# criterion = criterion.to(device)
# def binary_accuracy(preds, y):
#     #round predictions to the closest integer
#     rounded_preds = torch.round(torch.sigmoid(preds))
#     correct = (rounded_preds == y).float() 
#     acc = correct.sum() / len(correct)
#     return acc
# def train(model, iterator, optimizer, criterion):
#     epoch_loss = 0
#     epoch_acc = 0
#     model.train()  
#     for batch in iterator:
#         optimizer.zero_grad()   
#         text, text_lengths = batch.text
#         predictions = model(text, text_lengths).squeeze(1)    # for second
#         loss = criterion(predictions, batch.label)        
#         acc = binary_accuracy(predictions, batch.label)   
#         loss.backward()       
#         optimizer.step()      
#         epoch_loss += loss.item()  
#         epoch_acc += acc.item()    
#     return epoch_loss / len(iterator), epoch_acc / len(iterator)
# def evaluate(model, iterator, criterion):
#     epoch_loss = 0
#     epoch_acc = 0
#     model.eval()
#     with torch.no_grad():
#         for batch in iterator:
#             text, text_lengths = batch.text
#             predictions = model(text, text_lengths).squeeze(1)     # 1
#             loss = criterion(predictions, batch.label)
#             acc = binary_accuracy(predictions, batch.label)
#             epoch_loss += loss.item()
#             epoch_acc += acc.item()
#     return epoch_loss / len(iterator), epoch_acc / len(iterator)
# best_valid_loss = float('inf')
# train_losses = []
# val_losses = []
# for epoch in range(N_EPOCHS):
#     start_time = time.time()
#     train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
#     valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
#     end_time = time.time()
#     epoch_mins, epoch_secs = epoch_time(start_time, end_time)
#     if valid_loss < best_valid_loss:
#         best_valid_loss = valid_loss
#         torch.save(model.state_dict(), 'saved_weights.pt')
#     print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#     print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
#     print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%\n')
#     train_losses.append(train_loss)
#     val_losses.append(valid_loss)
# plt.plot(train_losses, label='Training loss')
# plt.plot(val_losses, label='Validation loss')
# plt.legend()
# plt.show()
# path='/content/saved_weights.pt'
# model.load_state_dict(torch.load(path));
# test_loss, test_acc = evaluate(model, valid_iterator, criterion)
# print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
# def predictFIRST(model, sentence, flag=False):
#     model.eval()
#     if(flag==False):
#       tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
#     else:
#         tokenized = sentence
#     indexed = [TEXT.vocab.stoi[t] for t in tokenized]
#     length = [len(indexed)]
#     tensor = torch.LongTensor(indexed).to(device)
#     tensor = tensor.unsqueeze(1)
#     length_tensor = torch.LongTensor(length)
#     prediction = torch.sigmoid(model(tensor, length_tensor))
#     return prediction.item()
# print("For text: """"This film is terrible"""" ->", predictFIRST(model, "This film is terrible"))
# print("For text: """"This film is great"""" ->", predictFIRST(model, "This film is great"))
# # neg
# print("For text: """"david carradine sad thai law sure fowl play"""" ->", predictFIRST(model, "david carradine sad thai law sure fowl play"))
# # pos
# print("For text: """"tell bro say congrats"""" ->", predictFIRST(model, "tell bro say congrats"))
# # pos
# print("For text: """"indeed"""" ->", predictFIRST(model, "indeed"))
# # pos
# print("For text: """"completed race life mins girlies work fun"""" ->", predictFIRST(model, "completed race life mins girlies work fun"))
# def writePreds(row):
#   if( round(predictFIRST(model, row))==1 ):
#     return "neg"
#   else:
#     return "pos"
# def computeDFAccuracy(df):
#   df['Preds'] = df['text'].apply(lambda x: writePreds(x))
#   df['Correct'] = df.apply(lambda x: x['label']==x['Preds'], axis = 1)
#   print(df['Correct'].unique())
#   print(df.Correct.value_counts()[True])
#   print(df.Correct.value_counts())
#   perCor = percentage(df.Correct.value_counts()[True], df.shape[0])
#   acc = accuracy_score(df['label'], df['Preds'])
#   return df, perCor, acc
# testTRY1, perCor, acc = computeDFAccuracy(testTRY1)
# print("Percentage of correct is ", perCor)
# print("Accuracy is ", acc)
# firstACC = acc
# end_timeTOTAL = time.time()
# mins, secs = epoch_time(start_timeTOTAL, end_timeTOTAL)
# print(f'Total Time: {mins}m {secs}s')

In [ ]:
!pip install torchtext==0.9

import torchtext

start_timeTOTAL = time.time()

TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)                   # for second

LABEL = torchtext.legacy.data.Field(dtype = torch.float, is_target=True, unk_token=None, batch_first=True)
# LABEL = data.LabelField(dtype = torch.float, is_target=True, unk_token=None, batch_first=True)

fields = [(None, None), ('label', LABEL), ('text',TEXT)]

training_data = torchtext.legacy.data.TabularDataset(
# training_data = data.TabularDataset(
                          path =  '/content/gdrive/My Drive/giannikos/trainTWEETS.csv',
                          format = 'csv',
                          fields = fields,
                          skip_header = True
                  )

# test_data = data.TabularDataset(
test_data = torchtext.legacy.data.TabularDataset(
                          path =  '/content/gdrive/My Drive/giannikos/testTWEETS.csv',
                          format = 'csv',
                          fields = fields,
                          skip_header = True
                  )

for i in range(5):
  print(vars(training_data.examples[i]))

defaultdict(<function _default_unk_index at 0x7f29e85c8ea0>, {'pos': 0, 'neg': 1})

oso pio konta sto 0 pio thetiko
oso pio konta sto 1 arnitiko

In [ ]:
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of test examples: {len(test_data)}')

In [ ]:
#initialize glove embeddings
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(
        train_data,
        max_size = MAX_VOCAB_SIZE,
        vectors = "glove.6B.100d",
        unk_init = torch.Tensor.normal_,
    )

LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))

untill=0
for item in TEXT.vocab.stoi.items():
    print(item)
    untill+=1
    if(untill==10):
      break

print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
      (train_data, valid_data, test_data), 
      batch_size = BATCH_SIZE,
      sort_key = lambda x: len(x.text),
      sort_within_batch=True,
      device = device
    )

In [ ]:
class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        embedded = embedded.permute(1, 0, 2)
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
                
        return self.fc(pooled)

In [ ]:
#define hyperparameters
INPUT_DIM       = len(TEXT.vocab)
EMBEDDING_DIM   = 100
N_FILTERS       = 100
HIDDEN_DIM      = 256
OUTPUT_DIM      = 1
N_LAYERS        = 2
BIDIRECTIONAL   = True
DROPOUT         = 0.5
N_EPOCHS        = 5
PAD_IDX         = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [ ]:
print(model)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

In [ ]:
optimizer = optim.Adam(model.parameters())
# optimizer = optim.RMSprop(model.parameters())
criterion = nn.BCEWithLogitsLoss()
# criterion = nn.MSELoss()

In [ ]:
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

#define metric
def binary_accuracy(preds, y):

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)

    return acc

There are 2 phases while building the model:

* Training phase: model.train() sets the model on the training phase and activates the dropout layers.
* Inference phase: model.eval() sets the model on the evaluation phase and deactivates the dropout layers.

Note, you do not need to do model.forward(batch.text), simply calling the model works.

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        predictions = model(batch.text).squeeze(1)              # for third, second
        #compute the loss
        loss = criterion(predictions, batch.label)        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        #backpropage the loss and compute the gradients
        loss.backward()       
        #update the weights
        optimizer.step()      
        
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
            
            predictions = model(batch.text).squeeze(1)                # for third, second
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

we will train the model for a certain number of epochs and save the best model every epoch

In [ ]:
best_valid_loss = float('inf')

train_losses = []
val_losses = []

for epoch in range(N_EPOCHS):

    start_time = time.time()
     
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%\n')

    train_losses.append(train_loss)
    val_losses.append(valid_loss)


plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

Τhe metric we actually care about, the test loss and accuracy, which we get from our parameters that gave us the best validation loss.

load the best model and define the inference function  that accepts the user defined input and make predictions

In [ ]:
#load weights
path='/content/saved_weights.pt'
model.load_state_dict(torch.load(path));

test_loss, test_acc = evaluate(model, valid_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
# second pred
def predictSECOND(model, sentence, flag=False):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
print("For text: """"This film is terrible"""" ->", predictSECOND(model, "This film is terrible"))
print("For text: """"This film is great"""" ->", predictSECOND(model, "This film is great"))
# neg
print("For text: """"david carradine sad thai law sure fowl play"""" ->", predictSECOND(model, "david carradine sad thai law sure fowl play"))
# pos
print("For text: """"tell bro say congrats"""" ->", predictSECOND(model, "tell bro say congrats"))

In [ ]:
# for i in range(20, 40, 1):
#   print( rejoin_words(vars(test_data.examples[i])['text']), '\t\t\tACTUAL\t',vars(test_data.examples[i])['label'], 'PREDICTED ',predictSECOND(model, rejoin_words(vars(test_data.examples[i])['text'])), "\n" )

In [ ]:
def writePredsSECOND(row):
  if( round(predictSECOND(model, row))==1 ):
    return "neg"
  else:
    return "pos"

def computeDFAccuracySECOND(df):

  df['Preds'] = df['text'].apply(lambda x: writePredsSECOND(x))
  df['Correct'] = df.apply(lambda x: x['label']==x['Preds'], axis = 1)

  print(df['Correct'].unique())
  print(df.Correct.value_counts()[True])
  print(df.Correct.value_counts())

  perCor = percentage(df.Correct.value_counts()[True], df.shape[0])
  acc = accuracy_score(df['label'], df['Preds'])

  return df, perCor, acc

testTRY2, perCor, acc = computeDFAccuracySECOND(testTRY2)

print("Percentage of correct is ", perCor)
print("Accuracy is ", acc)

secondACC = acc

testTRY2

In [ ]:
end_timeTOTAL = time.time()

mins, secs = epoch_time(start_timeTOTAL, end_timeTOTAL)

print(f'Total Time: {mins}m {secs}s')

In [ ]:
start_timeTOTAL = time.time()
TEXT = data.Field(tokenize = 'spacy', batch_first = True)                                   # third

LABEL = data.LabelField(dtype = torch.float, is_target=True, unk_token=None, batch_first=True)

fields = [(None, None), ('label', LABEL), ('text',TEXT)]

training_data = data.TabularDataset(
                          path =  '/content/gdrive/My Drive/DI/trainTWEETS.csv',
                          format = 'csv',
                          fields = fields,
                          skip_header = True
                  )

test_data = data.TabularDataset(
                          path =  '/content/gdrive/My Drive/DI/testTWEETS.csv',
                          format = 'csv',
                          fields = fields,
                          skip_header = True
                  )

for i in range(5):
  print(vars(training_data.examples[i]))

In [ ]:
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of test examples: {len(test_data)}')

In [ ]:
#initialize glove embeddings
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(
        train_data,
        max_size = MAX_VOCAB_SIZE,
        vectors = "glove.6B.100d",
        unk_init = torch.Tensor.normal_,
        # min_freq=4,
    )

LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

print(TEXT.vocab.freqs.most_common(10))

untill=0
for item in TEXT.vocab.stoi.items():
    print(item)
    untill+=1
    if(untill==10):
      break

print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
      (train_data, valid_data, test_data),
      batch_size = BATCH_SIZE,
      sort_key = lambda x: len(x.text),
      sort_within_batch=True,
      device = device
    )

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        return self.fc(cat)

In [ ]:
#define hyperparameters
INPUT_DIM     = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS     = 100
FILTER_SIZES  = [1, 1, 1]
HIDDEN_DIM    = 256
OUTPUT_DIM    = 1
N_LAYERS      = 2
BIDIRECTIONAL = True
DROPOUT       = 0.5
N_EPOCHS      = 5
PAD_IDX       = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
print(model)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

In [ ]:
optimizer = optim.Adam(model.parameters())
# optimizer = optim.RMSprop(model.parameters())

criterion = nn.BCEWithLogitsLoss()
# criterion = nn.MSELoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds))
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)

    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        predictions = model(batch.text).squeeze(1)              # for third
        #compute the loss
        loss = criterion(predictions, batch.label)        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        #backpropage the loss and compute the gradients
        loss.backward()       
        #update the weights
        optimizer.step()   
           
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
            
            predictions = model(batch.text).squeeze(1)                # for third
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

we will train the model for a certain number of epochs and save the best model every epoch

In [ ]:
best_valid_loss = float('inf')

train_losses = []
val_losses = []

for epoch in range(N_EPOCHS):

    start_time = time.time()
     
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%\n')

    train_losses.append(train_loss)
    val_losses.append(valid_loss)


plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

In [ ]:
#load weights
path='/content/saved_weights.pt'
model.load_state_dict(torch.load(path));

test_loss, test_acc = evaluate(model, valid_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
def predictTHIRD(model, sentence, flag=False, min_len = 5):
    model.eval()

    if(flag==False):
      tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    else:
      tokenized = sentence


    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
print("For text: """"This film is terrible"""" ->", predictTHIRD(model, "This film is terrible"))
print("For text: """"This film is great"""" ->", predictTHIRD(model, "This film is great"))
# neg
print("For text: """"david carradine sad thai law sure fowl play"""" ->", predictTHIRD(model, "david carradine sad thai law sure fowl play"))
# pos
print("For text: """"tell bro say congrats"""" ->", predictTHIRD(model, "tell bro say congrats"))

In [ ]:
# for i in range(20, 40, 1):
#   print( vars(test_data.examples[i])['text'], '\t\t\t\t\thave actual\t',vars(test_data.examples[i])['label'], 'and pred ', predictTHIRD(model, vars(test_data.examples[i])['text'], True)                )

In [ ]:
def writePredsTHIRD(row):
  if( round(predictTHIRD(model, row))==1 ):
    return "neg"
  else:
    return "pos"

def computeDFAccuracyTHIRD(df):

  df['Preds'] = df['text'].apply(lambda x: writePredsTHIRD(x))
  df['Correct'] = df.apply(lambda x: x['label']==x['Preds'], axis = 1)

  print(df['Correct'].unique())
  print(df.Correct.value_counts()[True])
  print(df.Correct.value_counts())

  perCor = percentage(df.Correct.value_counts()[True], df.shape[0])
  acc = accuracy_score(df['label'], df['Preds'])

  return df, perCor, acc

testTRY2, perCor, acc = computeDFAccuracyTHIRD(testTRY2)

print("Percentage of correct is ", perCor)
print("Accuracy is ", acc)

thirdACC = acc

testTRY2

In [ ]:
end_timeTOTAL = time.time()

mins, secs = epoch_time(start_timeTOTAL, end_timeTOTAL)

print(f'Total Time: {mins}m {secs}s')

In [ ]:
# print("First ACC", firstACC)
print("Second ACC", secondACC)
print("Third ACC", thirdACC)